In [10]:
import pandas as pd


In [ ]:
# these need to be connected to the tables

# Input variables
wind_speed_km_h = 22  # Wind load in km/h, from the shelter_info table: max_windspeed

# All the information from the shelters.xlsx
shelter_height = 3
shelter_width = 5
shelter_depth = 6
surface_area_m2 = shelter_depth * shelter_height  # Surface area in square meters, now assumed. In further development this could be calculated for each shelter. 
shelter_weight = 100

In [11]:
soil_df = pd.read_csv('data/soil_data.csv', sep=';')


ground_type = 8
soil_df

,Ground Type,Unnamed: 1,PSI,Bearing capacity (Kg/m2),Bearing capacity (kN/m2),c:cohesion Kpa = KN/m2,Angle of repose,Nc,Nq,Nγ,γ : unit weight of soil KN/m3,w:density of soil Kg/m2,
0,Rock,Bedrock,833,"585657,31","5742,702",10000,30,"37,2","22,5","19,7",26,3500,NaN
1,Rock,Layers,208,"146238,56","1433,952",10000,25,"25,1","12,7","9,7",24,2600,NaN
2,Rock,Soft,111,"78040,77","765,234",10000,20,"17,7","7,4",5,22,1690,NaN
3,"hardpan, cemented sand or gravel",all,139,"97726,73","958,266",0,40,"95,7","81,3","100,4",18,2000,NaN
4,gravel \r\nor sand,Compact,111,"78040,77","765,234",0,40,"95,7","81,3","100,4",21,2560,NaN
5,gravel \r\nor sand,Firm,83,"58354,81","572,202",0,35,"57,8","41,4","42,4",18,2000,NaN
6,gravel \r\nor sand,Loose,56,"39371,92","386,064",0,25,"25,1","12,7","9,7",16,1600,NaN
7,"Sand, coarse to medium",Compact,83,"58354,81","572,202",0,35,"57,8","41,4","42,4",19,1900,NaN
8,"Sand, coarse to medium",Firm,63,"44293,41","434,322",0,30,"37,2","22,5","19,7",17,1700,NaN
9,"Sand, coarse to medium",Loose,42,"29528,94","289,548",0,25,"25,1","12,7","9,7",16,1500,NaN


In [12]:
consistency_df = pd.read_csv('data/soil_consistency.csv',sep = ";", encoding='latin')
consistency_df

,SPT Penetrationï¿½(blows/ foot) (lower boundary),SPT Penetrationï¿½(blows/ foot) (upper boundary),Estimated Consistency,UCS(kPa) (lower boundary),UCS(kPa) (upper boundary)
0,0,2,Very Soft,0,24
1,2,4,Soft,24,48
2,4,8,Medium,48,96
3,8,15,Stiff,96,192
4,15,30,Very Stiff,192,388
5,30,100000,Hard,388,1000000


In [13]:
UCS = soil_df['c:cohesion Kpa = KN/m2'][18] * 2 # Unconfined Compressive Strength, UCs of the soiltype in the location (ref: https://eng.libretexts.org/Bookshelves/Civil_Engineering/Book%3A_The_Delft_Sand_Clay_and_Rock_Cutting_Model_(Miedema)/02%3A_Basic_Soil_Mechanics/2.04%3A_Soil_Mechanical_Parameters)

print(UCS)

est_consistency = []
foundation_type = []
for index, row in consistency_df.iterrows():
    if row['UCS(kPa) (lower boundary)'] <= UCS < row['UCS(kPa) (upper boundary)']:
        est_consistency = (row['Estimated Consistency'])
        print(est_consistency)
        if est_consistency == 'Hard':
            foundation_type = 'no foundation possible, use weights or pad layer'
        elif est_consistency in ('Very Stiff', 'Stiff'):
            foundation_type.extend(['base plate nailed to soil', 'pegs', 'screw in ground anchor'])
        elif est_consistency in ('Medium', 'Soft'):
            foundation_type.extend(['screw in ground anchor', 'embedded base plate', 'concrete bucket foundation'])
        if est_consistency in ('Soft', 'Very Soft'):
            foundation_type.append('concrete pad foundation')

foundation_type


20
Very Soft


['concrete pad foundation']

In [4]:
# Constants
air_density = 1.225  # kg/m^3 (standard air density)
conversion_factor = 1000 / 3600  # Conversion from km/h to m/s

# Convert wind speed from km/h to m/s
wind_speed_m_s = wind_speed_km_h * conversion_factor

# Calculate wind pressure
wind_pressure = 0.5 * air_density * wind_speed_m_s ** 2

# Calculate wind load (force) on the surface
wind_load_force = wind_pressure * surface_area_m2

# Output the results
print(f"Wind Speed (m/s): {wind_speed_m_s}")
print(f"Wind Pressure (Pa): {wind_pressure} Pa")
print(f"Wind Load Force (N): {wind_load_force} N")

Wind Speed (m/s): 6.111111111111112
Wind Pressure (Pa): 22.874228395061735 Pa
Wind Load Force (N): 457.4845679012347 N


In [40]:
# Calculation of uplift force, (heavily) simplified
# calculate the torque (momentum) caused by the windload: act like the building is a beam with one fixed side (B3 on the TU Delft structural mechanics formula sheet)
# 1/2ql**2
def uplift(wind_pressure, shelter_depth, shelter_height, shelter_width):
    q = wind_pressure * shelter_depth
    l = shelter_height
    torque = 1/2 * q*l**2
    torque

    # calculate the Fup and Fdown on the outsides of the shelter; T = R*F
    Fdown = torque/(1/2*shelter_width)
    Fup = -(Fdown)
    return Fup, Fdown

uplift_x = uplift(wind_pressure, shelter_depth, shelter_height, shelter_width)
uplift_y = uplift(wind_pressure, shelter_width,shelter_height,shelter_depth)

uplift_max = max(uplift_x + uplift_y)
uplift_min = min(uplift_x + uplift_y)

print(uplift_max, uplift_min)
anchor_needed = uplift_max - (shelter_weight*9.8)/(2)
if anchor_needed > 0:
    print('anchor needed:', anchor_needed, 'N')
else: 
    print('no anchor needed')

247.04166666666674 -247.04166666666674
no anchor needed
